In [ ]:
# Important: To get win32 working, we need to do the following steps
# Using pip:
# pip install pypiwin32
# Then open an administrator shell and do
# venv\Scripts\activate
# python venv\Scripts\pywin32_postinstall.py -install

import win32com.client as win32
import os
import psutil
import subprocess
import pandas as pd

from dataclasses import dataclass
from typing import List

In [ ]:
# Read input file
EXCEL_INPUT_FILE = "EC_SDS_TESTFILE.xlsx"

# Output file name for report creation
REPORT_FILENAME = "SDS_REMINDER_{}.xlsx"

# Set environment variables
OUTLOOK_APP_PATH = '"C:/Program Files/Microsoft Office/root/Office16/OUTLOOK.EXE"'
OUTLOOK_APP_NAME = "OUTLOOK.EXE"

# Other configuration parameters
MAILER_OPTIONS = {
    "AUTO_SEND": False  # Automatically sends mails if `True`. Set to `False` to create drafts instead.
}

# Sender email address. 
EMAIL_SENDER = "TODO@todo.com"


In [ ]:
# Define subject lines for mail here
MAILER_SUBJECT_JA = "最新版SDSの御提供のお願い [{}]"
MAILER_SUBJECT_EN = "Request for latest SDS [{}]"

# Define body text for mail here. Don't remove template strings wrapped inside `{}`!
MAILER_BODY_JA = """お取引先様各位

いつもお世話になっております。

現在弊社システムの定期的なメンテナンスを行っており、最新版のSDSを集めております。
つきましては貴社より購入しております、下記製品の和文と英文の両方のSDSの御提供をお願い致します。

{sds_details}

なお、以前に頂戴したSDSが弊社のシステムにあり記載されている改訂日が下記の通りです。

{sds_metadata}

改訂されていない場合はその旨お知らせいただければ幸いでございます。
お忙しいところ恐れ入りますが、ご協力のほど何卒よろしくお願い申し上げます。
"""

MAILER_BODY_EN = """To whom it may concern,

please provide BASF Totsuka with the latest Japanese and English SDS of your product(s).
Material(s) which we are purchasing, as follows:

{sds_details}

Revision date of SDS:

{sds_metadata}

Please let us know if there are no revised SDS.
Thank you for your cooperation.
"""


In [ ]:
# Email dataclass
@dataclass
class Email:
    sendFrom: str
    sendTo: str
    subject: str
    body: str


# Item dataclass
@dataclass
class Item:
    modifiedDate: str
    contactLanguage: str
    itemNo: str
    itemName: str


# Vendor dataclass
@dataclass
class Vendor:
    vendorName: str
    vendorEmail: str
    vendorLanguage: str
    items: List[Item]

    def createSubject(self) -> str:
        item_nos = "/".join(
            [f"#{item.itemNo}" for item in self.items]
        )
        # return subject according to vendorLanguage
        if self.vendorLanguage.upper() == "EN":
            message_subject = MAILER_SUBJECT_EN
        else:
            message_subject = MAILER_SUBJECT_JA
        return message_subject.format(item_nos)

    def createMessage(self) -> str:
        item_details = "\n".join(
            [
                f"{item.itemName} [#{item.itemNo}]"
                for item in self.items
            ]
        )
        item_metadata = "\n".join(
            [
                f"{item.contactLanguage} {item.modifiedDate.date()}"
                for item in self.items
            ]
        )

        sds_data = {
            "sds_details": item_details,
            "sds_metadata": item_metadata,
        }

        # return body according to vendorLanguage
        if self.vendorLanguage.upper() == "EN":
            message_body = MAILER_BODY_EN
        else:
            message_body = MAILER_BODY_JA
        return message_body.format(**sds_data)


In [ ]:
# Helper functions to retrieve data from excel sheet
def get_expired_df(df, expiry_date=None):
    """Takes a dataframe and an optional expiry_date, and returns
    a DataFrame of all series older than the given expiry_date.
    
    If no expiry_date is given, the current date will be used.
    """
    if not expiry_date:
        expiry_date = pd.Timestamp.today()

    try:
        expired = df[df.last_update < expiry_date]
        return expired
    except AttributeError as e:
        print(
            "Please check your input file contains a 'last_update' column.",
            e,
        )

def group_items_by_vendor(df):
    """Takes a dataframe and returns a dictionary
    with the vendor name and email as key,
    and a list of the items for that supplier as values. 
   
    Example:
        {('SAMPLE COMPANY', 'vendor@samplecompany.com', 'EN'): 
            [{'art': 'RX072299',
              'language': 'EN',
              'supplier': 'SAMPLE COMPANY',
              'last_update': Timestamp('2019-01-28 00:00:00'),
              'vendor_master.salutation': 'SAMPLE COMPANY',
              'vendor_master.email': 'vendor@samplecompany.com',
              'art_name': 'ARTICLE 123'}
            ]
        }
    """
    grouped = df.groupby(
        [
            "vendor_master.salutation",
            "vendor_master.email",
            "vendor_language",
        ]
    )
    group_dict = {}
    for name, group in grouped:
        if group_dict.get(name):
            group_dict[name].append(
                group.to_dict("records")
            )
        else:
            group_dict[name] = group.to_dict("records")
    return group_dict


def write_output_to_excel(df_out):
    """Takes a dataframe with item and vendor data
    and writes a local excel file to the desktop.
    """

    # Get current date and add to dataframe
    current_date = pd.Timestamp.today()
    filename = REPORT_FILENAME.format(
        current_date.strftime("%Y-%m-%d_%H-%M-%S")
    )
    df_out["current_date"] = current_date.date()

    # Write data to excel file
    writer = pd.ExcelWriter(filename, engine="xlsxwriter")
    df_out2 = df_out[["art", "art_name", "supplier", "current_date"]]
    df_out2.to_excel(writer, index=False, sheet_name="report")

    workbook = writer.book
    worksheet = writer.sheets["report"]

    # Adjust column widths
    worksheet.set_column("A:A", 15)
    worksheet.set_column("B:B", 30)
    worksheet.set_column("C:C", 60)
    worksheet.set_column("D:D", 15)
    writer.save()


In [ ]:
# Helper functions to handle API calls to Windows (needed to execute Outlook)
def open_outlook(path: str) -> None:
    """ Tries to open Outlook App from given path
    """
    try:
        subprocess.Popen(OUTLOOK_APP_PATH)
    except FileNotFoundError as e:
        print(
            f"Error: File not found at {OUTLOOK_APP_PATH}"
        )


def app_is_running(app_name: str) -> bool:
    """ Checks running processes to see if a given app is running.
    Returns:
        True -- if app is running else False
    """
    # Get names for all running processes
    running_processes = (
        psutil.Process(pid).name() for pid in psutil.pids()
    )

    for process in running_processes:
        if app_name in process:
            return True

    return False


def create_draft(mail: Email) -> None:
    """Takes an Email dataclass and saves a draft email.
    """
    outlook = win32.Dispatch("Outlook.Application")
    draft = outlook.CreateItem(0)
    draft.SentOnBehalfOfName = mail.sendFrom
    draft.To = mail.sendTo
    draft.Subject = mail.subject
    draft.body = mail.body
    draft.save()


def send_mail(mail: Email) -> None:
    """Takes an Email dataclass, creates a mail and sends it.
    """
    outlook = win32.Dispatch("Outlook.Application")
    draft = outlook.CreateItem(0)
    draft.SentOnBehalfOfName = mail.sendFrom
    draft.To = mail.sendTo
    draft.Subject = mail.subject
    draft.body = mail.body
    draft.send


def handle_mail(mail: Email) -> None:
    """Creates a draft or sends the email directly, depending on the app setting.
    """
    if MAILER_OPTIONS["AUTO_SEND"]:
        send_mail(mail)
    else:
        create_draft(mail)


In [ ]:
# Make sure we've got an email client running
if not app_is_running(OUTLOOK_APP_NAME):
    print("App not running, opening now.")
    open_outlook(OUTLOOK_APP_PATH)
else:
    print(f"{OUTLOOK_APP_NAME} is already running.")


In [ ]:
# Read input file as dataframe
excel_workbook = pd.read_excel(EXCEL_INPUT_FILE)


# Prepare a message queue
outbox = []

# Retrieve expired items, then group by supplier
expired_sds_df = get_expired_df(excel_workbook)
items_per_vendor = group_items_by_vendor(expired_sds_df)

# Transform data into Dataclasses
for vendor_info, items in items_per_vendor.items():
    # prepare item, supplier data for Vendor instance creation
    item_list = [
        Item(
            item.get("last_update"),
            item.get("language"),
            item.get("art"),
            item.get("art_name"),
        )
        for item in items
    ]
    vendor_name, vendor_email, vendor_language = vendor_info

    # create Vendor instance
    vendor = Vendor(
        vendorName=vendor_name,
        vendorEmail=vendor_email,
        vendorLanguage=vendor_language,
        items=item_list,
    )

    # Create combined emails per supplier
    email = Email(
        sendFrom=EMAIL_SENDER,
        sendTo=vendor.vendorEmail,
        subject=vendor.createSubject(),
        body=vendor.createMessage(),
    )
    outbox.append(email)

for mail in outbox:
    handle_mail(mail)

# Clear outbox when we're done sending
outbox = []

# Finally, save output as Excel file
try:
    write_output_to_excel(expired_sds_df)
    print("Report data saved!")
except Exception as e:
    print("Failed to save Excel file. Make sure the file isn't open already!")

In [ ]:
import win32gui


def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))


results = []
top_windows = []
win32gui.EnumWindows(windowEnumerationHandler, top_windows)
# print(top_windows)
for i in top_windows:
    if "Microsoft Outlook" in i[1]:
        win32gui.ShowWindow(i[0], 5)
        win32.Dispatch("WScript.Shell").SendKeys("%")
        win32gui.SetForegroundWindow(i[0])
